<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/bb/Apache_Hive_logo.svg/1200px-Apache_Hive_logo.svg.png" align ="left" width = 10%>
<h1><div class="alert alert-block alert-info">
Hadoop Hive
</div></h1>

The Apache Hive ™ data warehouse software facilitates reading, writing, and managing large datasets residing in distributed storage using SQL. Structure can be projected onto data already in storage. A command line tool and JDBC driver are provided to connect users to Hive.

<img src="https://3.bp.blogspot.com/-aZAMQztmmy0/WqbtksLeDpI/AAAAAAAACKg/hD8H0UsQGM8SulDBy1nR1Hr_Ra8IcTxSACLcBGAs/s640/hivearchitecture.png">

<h1><font color ='blue'>Practice in Hive</font></h1>

<b>- Configuring the environment:</b>

a) Upload the file that is going to be used:

scp driver.csv e.luizfonseca-dsti@edge-1.au.adaltas.cloud:

b) Transfering the file to HDFS

hdfs dfs -mkdir drivers_raw

hdfs dfs -put drivers.csv drivers_raw/drivers.csv

<b>- Connecting to Hive:</b>

beeline -u "jdbc:hive2://zoo-1.au.adaltas.cloud:2181,zoo-2.au.adaltas.cloud:2181,zoo-3.au.adaltas.cloud:2181/dsti;serviceDiscoveryMode=zooKeeper;zooKeeperNamespace=hiveserver2;"

# CSV File

<b>- Create an external table:</b>

CREATE EXTERNAL TABLE drivers_erika_csv (driverID int, name string, ssn string, location string, certified string, wageplan string)
row format delimited
fields terminated by ',' 
lines terminated by '\n' 
LOCATION 'user/e.luizfonseca-dsti/drivers_raw'
tblproperties ("skip.header.line.count"="1");

<b>- Importing data from CSV to table:</b>

load data inpath '/user/e.luizfonseca-dsti/drivers_raw/drivers.csv' into table drivers_erika_csv;

SELECT * FROM drivers_erika_csv LIMIT 20;

<b>- Create an external ORC table:</b>

CREATE EXTERNAL TABLE drivers_erika_ORC (driverID int, name string, ssn string, location string, certified string, wageplan string)
STORED AS ORC
location 'user/e.luizfonseca-dsti/drivers_raw';

<b>- Loading the data to the ORC table:</b>

INSERT INTO TABLE drivers_erika_ORC SELECT * FROM drivers_erika_csv;

### 💡 Creating an script to connect to Hive

<b>- Organizing the structure</b>

mkdir script

cd script/

<b>- Creating a file</b>

a) Creating a file

vi bee-connect.sh

write the beeline connection string (eg. beeline -u "jdbc:hive2://zoo-1.au.adaltas.cloud:2181,zoo-2.au.adaltas.cloud:2181,zoo-3.au.adaltas.cloud:2181/dsti;serviceDiscoveryMode=zooKeeper;zooKeeperNamespace=hiveserver2;") and save the file using :wq

b) Check if the file can be executed

ls -la

If it is not executable, change the permissions using the command:

chmod +x bee-connect.sh
cd ..

c) Test the connection

From now on we can use simple this command to load the connect with Hive.

./script/bee-connect.sh

# Internet Movie Database - IMDb

IMDB Tables:

- | imdb_name_basics              |

- | imdb_title_basics             |

- | imdb_title_crew               |

- | imdb_title_ratings            |

#### 1) Number of titles with duration superior than 2 hours.

SELECT COUNT(*) FROM imdb_title_basics WHERE RUNTIMEMINUTES > 120;

#### 2) Average duration of titles containing the string "world".

SELECT AVG(RUNTIMEMINUTES) FROM imdb_title_basics WHERE UCASE(primarytitle) LIKE "%WORLD%";

#### 3) Average rating of titles having the genre "Comedy"

SELECT avg(imdb_title_ratings.averagerating) 
    FROM imdb_title_ratings 
    JOIN imdb_title_basics ON imdb_title_basics.tconst = imdb_title_ratings.tconst 
    WHERE array_contains(imdb_title_basics.genres, "Comedy")

#### 4) Average rating of titles not having the genre "Comedy"
    
SELECT avg(imdb_title_ratings.averagerating) 
    FROM imdb_title_ratings 
    JOIN imdb_title_basics ON imdb_title_basics.tconst = imdb_title_ratings.tconst 
    WHERE NOT array_contains(imdb_title_basics.genres, "Comedy")


#### 5) Top 10 movies directed by Quentin Tarantino

#=======================================================================

<b> SOLUTION 1 - USING ARRAY_CONTAINS </b>
#=======================================================================

A) Filtering the movies that have Quentin Tarantino as director
SELECT imdb_title_crew.tconst FROM imdb_name_basics
    JOIN imdb_title_crew ON array_contains(imdb_title_crew.director, imdb_name_basics.nconst)
    WHERE imdb_name_basics.primaryname = 'Quentin Tarantino';

+-------------------------+
| imdb_title_crew.tconst  |
+-------------------------+
| tt3460252               |
| tt0378194               |
| tt0361748               |
| tt0320037               |
| tt0113101               |
| tt0266697               |
| tt0105236               |
| tt0119396               |
| tt0247082               |
| tt0108757               |
| tt0462322               |
| tt0110912               |
| tt0401792               |
| tt0359715               |
| tt1959459               |
| tt1853728               |
| tt6019206               |
| tt6493238               |
| tt1028528               |
| tt0615681               |
| tt7131622               |
| tt7713358               |
| tt0534695               |
| tt0568048               |
| tt0534696               |
+-------------------------+

B) Selecting the movies and it's rating
SELECT imdb_title_basics.tconst, imdb_title_basics.primarytitle, imdb_title_ratings.averagerating FROM imdb_title_basics JOIN imdb_title_ratings ON imdb_title_ratings.tconst = imdb_title_basics.tconst LIMIT 10;
    
C) Gathering all the information
SELECT imdb_title_basics.primarytitle, imdb_title_ratings.averagerating FROM imdb_title_basics
    JOIN imdb_title_ratings ON imdb_title_basics.tconst = imdb_title_ratings.tconst
    WHERE imdb_title_basics.tconst IN (SELECT imdb_title_crew.tconst FROM imdb_name_basics
    JOIN imdb_title_crew ON array_contains(imdb_title_crew.director, imdb_name_basics.nconst)
    WHERE imdb_name_basics.primaryname = 'Quentin Tarantino') ORDER BY imdb_title_ratings.averagerating DESC LIMIT 10;

In [ ]:
+-------------------------------------+-----------------------------------+
|   imdb_title_basics.primarytitle    | imdb_title_ratings.averagerating  |
+-------------------------------------+-----------------------------------+
| Pulp Fiction                        | 8.9                               |
| Kill Bill: The Whole Bloody Affair  | 8.8                               |
| Grave Danger: Part 2                | 8.6                               |
| Grave Danger: Part 1                | 8.6                               |
| Django Unchained                    | 8.4                               |
| Reservoir Dogs                      | 8.3                               |
| Inglourious Basterds                | 8.3                               |
| Kill Bill: Vol. 1                   | 8.1                               |
| Kill Bill: Vol. 2                   | 8.0                               |
| Sin City                            | 8.0                               |
+-------------------------------------+-----------------------------------+

10 rows selected (54.003 seconds)

#=======================================================================

<b> SOLUTION 2 - USING EXPLODE </b>
#=======================================================================


A) Exploding the array to have one column with the expanded director in rows
SELECT * FROM (SELECT * FROM imdb_title_crew LATERAL VIEW explode(director) exp as ndir) as exp_crew LIMIT 10;

B) Compiling all the information

SELECT imdb_title_basics.primarytitle, imdb_title_ratings.averagerating 
    FROM (SELECT * FROM imdb_title_crew LATERAL VIEW explode(director) exp as ndir) as exp_crew
    JOIN imdb_name_basics ON exp_crew.ndir = imdb_name_basics.nconst
    JOIN imdb_title_basics ON exp_crew.tconst = imdb_title_basics.tconst
    JOIN imdb_title_ratings ON imdb_title_ratings.tconst = imdb_title_basics.tconst
    WHERE imdb_name_basics.primaryname = "Quentin Tarantino"
    ORDER BY imdb_title_ratings.averagerating DESC
    LIMIT 10;

10 rows selected (30.029 seconds)

💡 Using EXPLODE was far away more efficient than using ARRAY_CONTAINS.